In [1]:
import pickle
import warnings

import numpy as np
import pandas as pd
#import psycopg2

from utils import ml_utils, db_utils

warnings.filterwarnings("ignore")

In [3]:
conn, engine = db_utils.connect('dev')

In [24]:
conn.close()

***
## 初期登録レコードを作成

In [2]:
columns = (
    'id', 'pregnancies', 'glucose', 'blood_pressure', 'skin_thickness', 
    'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'outcome'
)
source_df = pd.read_csv('./data/train.csv', header=0, names=columns)

In [3]:
source_df.head()

,id,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome
0,200,9,125,74,0,0,28.536910,0.444902,45,1
1,3832,4,109,80,0,0,28.047673,0.238243,22,0
2,4927,4,88,78,39,0,52.371341,0.279471,26,0
3,4088,9,125,74,0,0,40.062688,0.203922,45,0
4,3644,5,107,78,44,284,52.935068,0.284959,45,1


In [5]:
# 欠損値が0として入っているので直す
source_df["blood_pressure"] = source_df["blood_pressure"].apply(lambda x: np.nan if x == 0 else x)
source_df["skin_thickness"] = source_df["skin_thickness"].apply(lambda x: np.nan if x == 0 else x)
source_df["insulin"] = source_df["insulin"].apply(lambda x: np.nan if x == 0 else x)
source_df["bmi"] = source_df["bmi"].apply(lambda x: np.nan if x < 1 else x)

train_df = source_df.iloc[0:1500, :]
input_df = source_df.iloc[1500:2500, :]
test_df = source_df.iloc[2500:3000, :]
train_df['is_trained'] = True
input_df['is_trained'] = False
train_df.to_csv('data/init_train_data.csv', index=False, header=False)
input_df.to_csv('data/init_input_data.csv', index=False, header=False)
test_df.to_csv('data/init_test_data.csv', index=False, header=False)

In [7]:
train_df = pd.read_csv('./data/init_train_data.csv')
train_df.head()

,index,pregnancies,glucose,blood_pressure,skin_thickness,insulin,bmi,diabetes_pedigree_function,age,outcome,is_trained
0,200,9,125,74.0,NaN,NaN,28.536910,0.444902,45,1,True
1,3832,4,109,80.0,NaN,NaN,28.047673,0.238243,22,0,True
2,4927,4,88,78.0,39.0,NaN,52.371341,0.279471,26,0,True
3,4088,9,125,74.0,NaN,NaN,40.062688,0.203922,45,0,True
4,3644,5,107,78.0,44.0,284.0,52.935068,0.284959,45,1,True


In [20]:
# diabetes_diagnosis_resultsテーブル作成
sql = """
    DROP TABLE IF EXISTS diabetes_diagnosis_results;
    CREATE TABLE diabetes_diagnosis_results(
        index INTEGER,
        pregnancies INTEGER,
        glucose INTEGER,
        blood_pressure NUMERIC,
        skin_thickness NUMERIC,
        insulin NUMERIC,
        bmi NUMERIC,
        diabetes_pedigree_function NUMERIC,
        age INTEGER,
        outcome INTEGER,
        is_trained BOOLEAN,
        primary key (index)
    );
    """
db_utils.execute(conn, sql)

In [21]:
# diabetes_diagnosis_resultsテーブルに初期データロード
table_name = 'diabetes_diagnosis_results'
train_df.to_sql(table_name, con=engine, if_exists="append", index=False)

In [23]:
# diabetes_diagnosis_resultsテーブルの件数確認
sql = """
    select count(*) from diabetes_diagnosis_results where is_trained = True;
"""
db_utils.fetch_all(conn, sql)

[(1500,)]

***
## 初期trainデータでモデル作成後、初期inputデータを入れる

In [8]:
# 初期ユーザー入力データを入れる
table_name = 'diabetes_diagnosis_results'
input_df.to_sql(table_name, con=engine, if_exists="append", index=False)

In [10]:
# diabetes_diagnosis_resultsテーブルの件数確認
sql = """
    select count(*) from diabetes_diagnosis_results where is_trained = False;
"""
db_utils.fetch_all(conn, sql)

[(1000,)]

***

***
## predict_resultテーブルに初期レコードを入れる

In [13]:
# predict_resultsテーブル作成
sql = """
    DROP TABLE IF EXISTS predict_results;
    CREATE TABLE predict_results(
        index INTEGER,
        predict_result INTEGER,
        predict_probability NUMERIC,
        true_result INTEGER,
        model_id INTEGER,
        primary key (index)
    );
    """
db_utils.execute(conn, sql)

In [18]:
test_y

1500    False
1501    False
1502    False
1503    False
1504    False
        ...  
2495    False
2496    False
2497    False
2498    False
2499    False
Name: is_trained, Length: 1000, dtype: bool

In [19]:
# predict_resultにサンプルデータを挿入
imputer_id = "a959d262dfc95a584d17637673c23395"
model_id = "7d28bf6a69a6398a187e5c2c47cfdcba"
columns = ['index', 'pregnancies', 'glucose', 'blood_pressure', 'skin_thickness',
           'insulin', 'bmi', 'diabetes_pedigree_function', 'age', 'outcome']
df = input_df[columns]
test_X, test_y = df.iloc[:, :-1], df.iloc[:, -1]
test_X.drop("index", axis=1, inplace=True)
imputer = pickle.load(open(f"./resources/deploy/model/imputer_{imputer_id}.pkl", "rb"))
test_X = pd.DataFrame(imputer.transform(test_X), columns=test_X.columns)
model = pickle.load(open(f"./resources/deploy/model/model_{model_id}.pkl", "rb"))
test_y_pred = model.predict(test_X)
test_y_proba = model.predict_proba(test_X)[:, 1]

In [21]:
predict_result_df = pd.DataFrame()
predict_result_df['index'] = df['index']
predict_result_df['predict_result'] = test_y_pred
predict_result_df['predict_probability'] = test_y_proba
predict_result_df['true_result'] = test_y
predict_result_df['model_id'] = model_id
predict_result_df.head()

,index,predict_result,predict_probability,true_result,model_id
1500,907,0,0.077556,0,7d28bf6a69a6398a187e5c2c47cfdcba
1501,3679,0,0.181595,0,7d28bf6a69a6398a187e5c2c47cfdcba
1502,3167,0,0.120691,0,7d28bf6a69a6398a187e5c2c47cfdcba
1503,1628,0,0.111972,1,7d28bf6a69a6398a187e5c2c47cfdcba
1504,102,0,0.340563,1,7d28bf6a69a6398a187e5c2c47cfdcba


In [22]:
predict_result_df.to_sql('predict_results', con=engine, if_exists='replace', index=False)

In [24]:
# 確認
sql = """
    select count(*) from predict_results;
"""
db_utils.fetch_all(conn, sql)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [8]:
# results_tempテーブル作成
sql = """
    DROP TABLE IF EXISTS results_temp;
    CREATE TABLE results_temp(
        index INTEGER,
        pregnancies INTEGER,
        glucose INTEGER,
        blood_pressure INTEGER,
        skin_thickness INTEGER,
        insulin INTEGER,
        bmi NUMERIC,
        diabetes_pedigree_function NUMERIC,
        age INTEGER,
        predict_result INTEGER,
        predict_probability NUMERIC,
        primary key (index)
    );
    """
db.execute(conn, sql)

In [4]:
# 確認
sql = """
    select count(1) from results_temp;
"""
db_utils.fetch_all(conn, sql)

[(1,)]

In [23]:
conn.close()

In [11]:
predict_result_df = pd.read_sql(sql="SELECT * FROM predict_results;", con=conn)
predict_result_df

,index,predict_result,predict_probability,true_result,model_id
0,114,0,0.108578,0,20220110115017
1,1816,0,0.086963,0,20220110115017
2,1594,1,0.750121,1,20220110115017


In [12]:
from sklearn.metrics import accuracy_score, auc, roc_curve
userinput_y = predict_result_df["true_result"].values
userinput_y_score = predict_result_df["predict_probability"].values
userinput_fpr, userinput_tpr, _ = roc_curve(userinput_y, userinput_y_score)
userinput_fpr, userinput_tpr

(array([0., 0., 1.]), array([0., 1., 1.]))

In [8]:
input_dict = {
    "subject_id": [2],
    "pregnancies": [3],
    "glucose": [4],
    "bloodpressure": [5],
    "skinthickness": [6],
    "insulin": [7],
    "bmi": [8],
    "diabetespedigreefunction": [9],
    "age": [10],
}
input_df = pd.DataFrame(input_dict)
input_df["predict_result"] = 11
input_df["predict_probability"] = 12

In [9]:
input_df

,subject_id,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,predict_result,predict_probability
0,2,3,4,5,6,7,8,9,10,11,12


In [12]:
input_df.loc[0, 'subject_id'] = 8

In [13]:
input_df

,subject_id,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,predict_result,predict_probability
0,8,3,4,5,6,7,8,9,10,11,12


In [10]:
input_df.to_sql("results_temp", con=engine, if_exists="append", index=False)

ProgrammingError: (psycopg2.ProgrammingError) column "subject_id" of relation "results_temp" does not exist
LINE 1: INSERT INTO results_temp (subject_id, pregnancies, glucose, ...
                                  ^

[SQL: INSERT INTO results_temp (subject_id, pregnancies, glucose, bloodpressure, skinthickness, insulin, bmi, diabetespedigreefunction, age, predict_result, predict_probability) VALUES (%(subject_id)s, %(pregnancies)s, %(glucose)s, %(bloodpressure)s, %(skinthickness)s, %(insulin)s, %(bmi)s, %(diabetespedigreefunction)s, %(age)s, %(predict_result)s, %(predict_probability)s)]
[parameters: {'subject_id': 2, 'pregnancies': 3, 'glucose': 4, 'bloodpressure': 5, 'skinthickness': 6, 'insulin': 7, 'bmi': 8, 'diabetespedigreefunction': 9, 'age': 10, 'predict_result': 11, 'predict_probability': 12}]
(Background on this error at: http://sqlalche.me/e/f405)